

# <center>Regresión Lineal con TensorFlow</center>
**Julio 2020** <br>
**Intructor:** Eduardo Marín Nicolalde

A continuación, analizaremos el funcionamiento  de `TensorFlow` entrenando una regresión lineal simple. Para ello seguiremos un conjunto de pasos que tienen por objetivo  comprender las etapas de entrenamiento de un modelo de forma general.


In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

**Dataset**

Emplearemos Datos sintéticos para ejemplificar el uso de ``Keras`` en ``TensorFlow``

In [0]:
feature   = ([1.0, 2.0,  3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0, 10.0, 11.0, 12.0])
label     = ([5.0, 8.8,  9.6, 14.2, 18.8, 19.5, 21.4, 26.8, 28.9, 32.0, 33.8, 38.2])

**Etapas**

1) Los modelos más simples de ``tf.keras`` son de tipo **secuencial**. Este agrupa capas de forma lineal.
* Un modelo secuencial contiene al menos una capa

[1. Documentación modelo secuencial](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)

[2. Glosario de términos](https://developers.google.com/machine-learning/glossary/)

In [0]:
# Seleccionar el tipo de modelo/red
model = tf.keras.models.Sequential()

2) Se describe la topografía del modelo. 
* Se añade una capa `model.add` densamente conectada `tf.keras.layers.dense`
* En el caso de una regresión lineal, se emplea una capa `layers` y una neurona `units`.
* Cuando trabajamos con la primera capa, es necesario especificar `input_shape`, la forma de la capa de entrada.

In [0]:
# Topografía de regresión lineal
model.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))

3) Compilar la topografía del modelo `compile` de forma que `TensorFlow` lo pueda ejecutar de forma eficiente.
* Se selecciona el método de optimización `optimizer` para minimizar la función `loss`, así como la tasa de aprendizaje `lr`
* Se selecciona la función `loss` de acuerdo al problema a resolver (regresión, clasificación, entre otros...)
* Se selecciona la métrica `metrics` para evaluar el ajuste en entrenamiento y prueba (train & test)

In [0]:
# Compilación
my_learning_rate = 0.01 
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss      = "mean_squared_error",
                metrics   = [tf.keras.metrics.RootMeanSquaredError()])

4) Una vez especificada la arquitectura de la red, se realiza el ajuste del modelo `fit`
* Se determina los predictores y la variable objetivo `x` e `y`
* La red puede entrenarse con subconjuntos de datos en diferentes iteraciones del proceso de optimización [batch](https://developers.google.com/machine-learning/glossary/#batch). Se define el tamaño de esos subgrupos por medio de [batch_size].(https://developers.google.com/machine-learning/glossary/#batch-size)
* Se especifica el número de veces que se analiza el dataset completo. Un [epoch](https://developers.google.com/machine-learning/glossary/#epoch) representa `N/batch_size` iteraciones de entrenamiento, donde `N` es el número de ejemplos en el dataset.

In [0]:
# Entrenamiento 
learning_rate = 0.01
epochs        = 10
my_batch_size = 6

history = model.fit(x=feature,
                    y=label,
                    batch_size=my_batch_size,
                    epochs=epochs)

5) Obtener los coeficientes de la regresión lineal simple
* El intercepto también es conocido como `bias`
* El parametro beta 1 es denominado peso o `weight`

In [0]:
# Parámetros de la regresión
trained_weight = model.get_weights()[0]
trained_bias   = model.get_weights()[1]

print("beta 1 es igual a: ", trained_weight)
print("beta 0 es igual a: ", trained_bias)

In [0]:
model.get_weights()

In [0]:
# Evolución del proceso de optimización
hist = pd.DataFrame(history.history)
hist

In [0]:
# Épocas entrenadas y evolución del error
epochs = history.epoch
rmse   = hist["root_mean_squared_error"]

6) Definimos 2 funciones para graficar:
* Gráfico de regresión lineal simple
* La evolución del entrenamiento del modelo

In [0]:
# 1. Grafico de regresión lineal simple
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against the training feature and label."""

  # Label the axes.
  fig, ax = plt.subplots()
  ax.plot(feature, label, 'o')
  ax.plot(feature, *trained_weight*feature + trained_bias)
  
  plt.xlabel("feature")
  plt.ylabel("label")


  
# 2. Evolución del entrenamiento del modelo
def plot_the_loss_curve(epochs, rmse):
  """Plot the loss curve, which shows loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()

In [0]:
plot_the_model(trained_weight, trained_bias, feature, label)

In [0]:
plot_the_loss_curve(epochs, rmse)

**¿Qué puede comentar sobre el ajuste del modelo?**

## 4. Automatizando el modelo
Las siguientes funciones permiten automatizar el proceso de entrenamiento del modelo, seleccionando distintos hiperparámetros.


In [0]:
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential. 
  # A sequential model contains one or more layers.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer. 
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compile the model topography into code that 
  # TensorFlow can efficiently execute. Configure 
  # training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model           


def train_model(model, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the feature values and the label values to the 
  # model. The model will train for the specified number 
  # of epochs, gradually learning how the feature values
  # relate to the label values. 
  history = model.fit(x=feature,
                      y=label,
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the 
  # rest of history.
  epochs = history.epoch
  
  # Gather the history (a snapshot) of each epoch.
  hist = pd.DataFrame(history.history)

  # Specifically gather the model's root mean 
  #squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

La siguiente celda de código automatiza el proceso de entrenamiento. Juegue con los hiperparámetros y encuentre una solución semióptima para el problema de regresión.

### 4.1 Tarea 1: Incremente el número de épocas

El `training loss` debe disminuir de manera constante, abruptamente al principio, y luego más lentamente. Finalmente, el `training loss` debería mantenerse estable (pendiente cero o pendiente casi cero), lo que indica que el entrenamiento ha convergido.

En el ejemplo de la sección 3, el `training loss` no convergió. Una posible solución es entrenar para más épocas. Aumente el número de épocas lo suficiente como para lograr que el modelo converja. 

Sin embargo, es ineficiente entrenar la convergencia pasada, así la tarea no consiste en simplemente establecer un número alto de épocas. 

Examina la curva de pérdida. ¿El modelo converge?

In [0]:
# Configuración de hiperparámetros
learning_rate   = 0.1
my_epochs       = ? #reemplaze ? por un número entero
my_batch_size   = 12

#Entrenamiento del modelo
m2                                         = build_model(my_learning_rate = learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(model = m2, feature = feature , label = label, epochs = my_epochs, batch_size = my_batch_size)

# Plot de resultados
plot_the_model(trained_weight, trained_bias, feature, label)
plot_the_loss_curve(epochs, rmse)

In [0]:
print("beta 1 es igual a: ", trained_weight)
print("beta 0 es igual a: ", trained_bias)

### 4.2 Tarea 2: Incremente la tasa de aprendizaje

En la Tarea 1, usted aumentó el número de épocas para lograr que el modelo converja. A veces, puede lograr que el modelo converja más rápidamente al aumentar la tasa de aprendizaje. Sin embargo, establecer la tasa de aprendizaje demasiado alta a menudo hace imposible que un modelo converja. En la Tarea 2, hemos establecido intencionalmente la tasa de aprendizaje demasiado alta. Ejecute la siguiente celda de código y vea qué sucede.



In [0]:
# Configuración de hiperparámetros
learning_rate   = 0.1
my_epochs       = 10
my_batch_size   = 12

#Entrenamiento del modelo
m3                                         = build_model(my_learning_rate = learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(model = m3, feature = feature , label = label, epochs = my_epochs, batch_size = my_batch_size)

# Plot de resultados
plot_the_model(trained_weight, trained_bias, feature, label)
plot_the_loss_curve(epochs, rmse)

El modelo resultante es terrible; la línea de regresión no se alinea con los puntos azules. Además, la curva de pérdida oscila como una montaña rusa. Una curva de pérdida oscilante sugiere fuertemente que la tasa de aprendizaje es demasiado alta.

### 4.3 Tarea 3: Busque la combinación ideal de epochs y learning rate

Asigne valores a los siguientes dos hiperparámetros para que el entrenamiento converja de la manera más eficiente posible:
* learning_rate
* epochs


In [0]:
# Configuración de hiperparámetros
learning_rate   = ? #Reemplace por un número decimal
my_epochs       = ?  #Reemplace por un número entero
my_batch_size   = 12

#Entrenamiento del modelo
m3                                         = build_model(my_learning_rate = learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(model = m3, feature = feature , label = label, epochs = my_epochs, batch_size = my_batch_size)

# Plot de resultados
plot_the_model(trained_weight, trained_bias, feature, label)
plot_the_loss_curve(epochs, rmse)

### 4.4 Tarea 4: Ajuste el batch_size

El sistema recalcula el valor de pérdida del modelo `loss` y ajusta los pesos y el sesgo del modelo después de cada **iteración**. Cada iteración es el espacio en el que el sistema procesa un batch. 

Por ejemplo, si el tamaño del batch es 6, el sistema vuelve a calcular el valor de ``loss`` del modelo y ajusta los pesos y el sesgo del modelo cada 6 ejemplos/observaciones procesados.

Una ``epoch`` abarca suficientes iteraciones para procesar cada ejemplo en el conjunto de datos. Por ejemplo, si el tamaño del batch es **12**, cada época dura una iteración. Sin embargo, si el tamaño del batch es 6, cada época consume dos iteraciones.

Pensaríamos que el ejercicio implica simplemente establecer el tamaño del batch en la cantidad de ejemplos en el conjunto de datos (12, en este caso). Sin embargo, el modelo podría entrenar más rápido en batchs más pequeños. Por el contrario, los batchs muy pequeños pueden no contener suficiente información para ayudar a que el modelo converja.

Experimente con ``batch_size`` en la siguiente celda de código. ¿Cuál es el número entero más pequeño que puede establecer para batch_size y aún así hacer que el modelo converja en cien épocas?


In [0]:
# Configuración de hiperparámetros
learning_rate   = 0.05
my_epochs       = 20
my_batch_size   = ? #reemplace con un número entero

#Entrenamiento del modelo
m4                                         = build_model(my_learning_rate = learning_rate)
trained_weight, trained_bias, epochs, rmse = train_model(model = m4, feature = feature , label = label, epochs = my_epochs, batch_size = my_batch_size)

# Plot de resultados
plot_the_model(trained_weight, trained_bias, feature, label)
plot_the_loss_curve(epochs, rmse)

Compare sus resultados con el estimador de mínimos cuadrados a continuación:

In [0]:
y = np.matrix([label])
x = np.matrix([np.ones(12),
               feature])
y * x.T * np.linalg.inv(x*x.T)

## 5. Sumario de ajuste de hiperparámetros

La mayoría de los problemas de aprendizaje automático requieren una gran cantidad de ajustes de hiperparámetros. Desafortunadamente, no podemos proporcionar reglas de ajuste concretas para cada modelo.

Bajar la tasa de aprendizaje puede ayudar a que un modelo converja de manera eficiente, pero hace que la convergencia sea demasiado lenta. 

Se debe experimentar para encontrar el mejor conjunto de hiperparámetros para su conjunto de datos. Dicho esto, aquí hay algunas reglas generales:

* La función `loss` debería disminuir de manera constante, abruptamente al principio, y luego más lentamente hasta que la pendiente de la curva alcance o se acerque a cero.
* Si la función `loss` no converge, entrena para más épocas.
* Si la función `loss` disminuye demasiado lento, aumente la tasa de aprendizaje. Tenga en cuenta que establecer la `loss` demasiado alta también puede evitar la convergencia.
* Si la función `loss` varía enormemente (es decir, la pérdida de entrenamiento salta), disminuya la tasa de aprendizaje.
* Reducir la tasa de aprendizaje al tiempo que aumenta el número de épocas o el tamaño del lote suele ser una buena combinación.
* Establecer el tamaño del batch en un número de muy pequeño también puede causar inestabilidad. Primero, pruebe valores de tamaño de batch grande. Luego, disminuya el tamaño del batch hasta que vea degradación.
* Para los conjuntos de datos del mundo real que consisten en una gran cantidad de ejemplos, el conjunto de datos completo podría no caber en la memoria. En tales casos, deberá reducir el tamaño del batch para permitir que un batch se ajuste a la memoria.

**Recuerde: la combinación ideal de hiperparámetros depende de los datos, por lo que siempre debe experimentar y verificar.**

